In [12]:
import os
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
import numpy as np

In [14]:
enable_prallel_processing = 0 # 0-disable 1-enable
no_sub_to_run_in_parallel = 4
sub_det_df = pd.read_excel('data/rs_fmri_sub_details_WO_AS.xlsx')
cur_sub_list_before = sub_det_df['Subject_ID_before'].to_numpy()
cur_sub_list_after = sub_det_df['Subject_ID_after'].to_numpy()
subject_ids = np.concatenate((cur_sub_list_before,cur_sub_list_after))
print('# of subjects to Process:', len(subject_ids))

# of subjects to Process: 178


In [2]:
FS_dir = '/data-braingut/bharath/anat_freesurfer_V6'
base_dir = '/data-braingut/bharath/nifty_used_data'

function to run freesurfer recon-all

In [4]:
def run_reconn_all(subid, base_dir, FS_dir):
    cmdline = 'recon-all -all -T2 %s/%s/anat/sub-%s_T2w.nii.gz -openmp 10 -parallel -subjid %s -sd %s -T2pial -i %s/%s/anat/sub-%s_T1w.nii.gz'%(base_dir,subid,subid, subid, FS_dir, base_dir,
                                                                                                                                                subid, subid)
#     print(cmdline)
    os.system(cmdline)


In [28]:
if(enable_prallel_processing == 1):
    with ProcessPoolExecutor(max_workers = no_sub_to_run_in_parallel) as executor:
        for i in subject_ids:
            executor.submit(run_reconn_all, i, base_dir, FS_dir)  
else:
    for i in subject_ids:
        run_reconn_all(i, base_dir, FS_dir)  

function to convert freesurfer ouput to afni usable format

In [8]:
def run_Make_Spec_FS(sub_id, FS_dir):
    cmdline = (r'@SUMA_Make_Spec_FS -NIFTI -fspath %s/%s/ -sid %s') % (FS_dir, sub_id,sub_id)
#     print(cmdline)
    os.system(cmdline)

In [ ]:
if(enable_prallel_processing == 1):
    with ProcessPoolExecutor(max_workers=no_sub_to_run_in_parallel*8) as executor:
        for i in subject_ids:
            executor.submit(run_Make_Spec_FS, i,FS_dir)
else:
    for i in subject_ids:
        run_Make_Spec_FS(i, FS_dir)